### This is an implementation of the Gale-Shapley algorithm for the stable marriage problem

In [4]:
import numpy as np
import random
from itertools import permutations,product

In [5]:
def reverse(preferences):
    n= len(preferences)
    return [preferences.index(i) for i in range(n)]

In [8]:
def gale_shapley(p_prefs, d_prefs):## This function takes the matrix preferences of the proponents and disponents.
    ## it returns a list d_partners whose i-th component is the partner of the i-th disponent
    n = len(p_prefs)
    d_partner = [-1] * n
    p_free = [True] * n
    free_count = n
    proposals=[[]]
    relationships=[[]]

    while free_count:
        m = p_free.index(True)
        for w in p_prefs[m]:
            if d_partner[w] == -1:
                d_partner[w] = m
                p_free[m] = False
                free_count -= 1
                proposals.append([(m,w)])
                proposals.append([])
                relationships.append(relationships[-1])
                relationships.append(relationships[-1]+[(m,w)])
                break
            else:
                m1 = d_partner[w]
                w_list = d_prefs[w]
                if w_list.index(m) < w_list.index(m1):
                    d_partner[w] = m
                    p_free[m] = False
                    p_free[m1] = True
                    proposals.append([(m,w)])
                    proposals.append([])
                    relationships.append(relationships[-1])
                    new_rel=relationships[-1]+[(m,w)]
                    new_rel.remove((m1,w))
                    relationships.append(new_rel)
                    break
                else:
                    proposals.append([(m,w)])
                    proposals.append([])
                    relationships.append(relationships[-1])
                    relationships.append(relationships[-1])

    return {"disponents_partners":d_partner,"proposals":proposals,"relationships":relationships}

## Random Example

In [10]:
k=10
men_pref=[]
women_pref=[]
for i in range(k):

    a=list(np.random.permutation(list(range(k))))
    #a=list(range(k))
    b=list(np.random.permutation(list(range(k))))
    men_pref.append(a)
    women_pref.append(b)

ans=gale_shapley(men_pref,women_pref)






In [12]:
rel=ans["relationships"]
pro=ans["proposals"]
final=ans["disponents_partners"]

In [13]:
result=sorted(rel[-1], key=lambda x: x[1])

In [14]:
result

[(2, 0),
 (6, 1),
 (8, 2),
 (4, 3),
 (7, 4),
 (5, 5),
 (0, 6),
 (9, 7),
 (1, 8),
 (3, 9)]

In [15]:
# this creates and instance of preferences
def make_preferences(n):
    ans=[]
    for i in range(n):
        ans.append(list(np.random.permutation(list(range(n)))))
    return ans

In [17]:
# This creates an instance of the problem and computes the solution
def make_instance(m):
    men_preferences = make_preferences(m)
    women_preferences = make_preferences(m)

    women_engagements = gale_shapley(men_preferences, women_preferences)["disponents_partners"]
    men_engagements=reverse(women_engagements)

    return men_preferences,women_preferences,men_engagements,women_engagements,

In [18]:
### This checks that a marriage is stable
def check_stable(men_pref,women_pref,men_eng,women_eng):
    n=len(men_pref)
    ans="Stable"
    for i in range(n):
        if ans=="Unstable":
            break
        for j in range(men_pref[i].index(men_eng[i])):
            woman_pref=women_pref[men_pref[i][j]]
            if woman_pref.index(i)<woman_pref.index(women_eng[men_pref[i][j]]):
                ans="Unstable"
                # print(men_pref[i])
                # print(women_pref[men_pref[i][j]])
                # print(men_eng)
                # print(women_eng)
                break
    return ans
    



## This is an experiment to check that check stable works well

In [20]:
k=5
res=[]
for i in range(1000):
    
    men_pref=[]
    women_pref=[]
    for i in range(k):
        men_pref=make_preferences(k)
        women_pref=make_preferences(k)

    ans=gale_shapley(men_pref,women_pref)
    a=ans["disponents_partners"]
    b=reverse(a)
    
    x=list(range(k))
    random.shuffle(x)
    y=reverse(x)
    T=check_stable(men_pref,women_pref,b,a)
    F=check_stable(men_pref,women_pref,x,y)
    res.append((T,F))




## This function finds all stable marriages given preference matrix

In [22]:
def find_all_stable_marriages(k):   
    men_pref=make_preferences(k)
    women_pref=make_preferences(k)

    stable_marriages=[]
    # Example list
    numbers=list(range(k))

    # Generate all permutations of the list
    possibilities = list(permutations(numbers))
    for possibility in possibilities:
        if check_stable(men_pref,women_pref,reverse(possibility),possibility)=="Stable":
            #print(possibility)
            stable_marriages.append(possibility)
    return {"men_pref":men_pref,"women_pref":women_pref,"stable_marriages":stable_marriages}


### Find an example whith many stable marriages

In [245]:
k=8
examples=[]
for i in range(500):
    examples.append(find_all_stable_marriages(k))

In [246]:
example=sorted(examples, key=lambda dictionary: len(dictionary["stable_marriages"]))[-1]

In [247]:
example

{'men_pref': [[0, 1, 4, 7, 6, 3, 2, 5],
  [1, 0, 4, 3, 2, 6, 5, 7],
  [5, 2, 6, 3, 7, 4, 0, 1],
  [2, 5, 3, 6, 0, 7, 1, 4],
  [7, 3, 0, 5, 6, 2, 1, 4],
  [3, 1, 5, 6, 7, 4, 0, 2],
  [4, 6, 5, 2, 0, 3, 1, 7],
  [6, 1, 7, 2, 5, 0, 3, 4]],
 'women_pref': [[3, 2, 1, 4, 7, 5, 0, 6],
  [5, 2, 0, 1, 4, 3, 7, 6],
  [5, 7, 3, 4, 2, 1, 0, 6],
  [6, 0, 1, 7, 3, 4, 5, 2],
  [3, 2, 7, 6, 4, 5, 1, 0],
  [3, 1, 7, 0, 6, 4, 2, 5],
  [6, 2, 4, 7, 0, 5, 3, 1],
  [0, 4, 6, 5, 7, 3, 1, 2]],
 'stable_marriages': [(0, 1, 3, 5, 6, 2, 7, 4),
  (0, 1, 7, 5, 2, 3, 6, 4),
  (0, 1, 7, 5, 6, 3, 2, 4),
  (1, 0, 3, 5, 6, 2, 7, 4),
  (1, 0, 7, 5, 2, 3, 6, 4),
  (1, 0, 7, 5, 6, 3, 2, 4),
  (1, 5, 3, 4, 6, 2, 7, 0),
  (1, 5, 7, 4, 2, 3, 6, 0),
  (1, 5, 7, 4, 6, 3, 2, 0)]}

In [248]:
men_pref=example["men_pref"]
women_pref=example["women_pref"]
sm=example["stable_marriages"]

## Check that marriages are stable

In [249]:
answers=[]
for a in sm:
    answers.append(check_stable(men_pref,women_pref,reverse(a),a))

In [29]:
answers

['Stable',
 'Stable',
 'Stable',
 'Stable',
 'Stable',
 'Stable',
 'Stable',
 'Stable']

### Compute sadness vector

In [250]:
def sadness_vectors(men_pref,women_pref,women_eng):

    a=women_eng
    b=reverse(a)
    n=len(a)
    women_sadness=[ women_pref[i].index(a[i])for i in range(n)]
    men_sadness=[men_pref[i].index(b[i])for i in range(n)]
    return {"men_sadness":men_sadness,"women_sadness":women_sadness}

In [251]:
k=3
print("men_pref",men_pref)
print("women_pref",women_pref)
print("women_eng",sm[k])
print("men_eng",reverse(sm[k]))

men_pref [[0, 1, 4, 7, 6, 3, 2, 5], [1, 0, 4, 3, 2, 6, 5, 7], [5, 2, 6, 3, 7, 4, 0, 1], [2, 5, 3, 6, 0, 7, 1, 4], [7, 3, 0, 5, 6, 2, 1, 4], [3, 1, 5, 6, 7, 4, 0, 2], [4, 6, 5, 2, 0, 3, 1, 7], [6, 1, 7, 2, 5, 0, 3, 4]]
women_pref [[3, 2, 1, 4, 7, 5, 0, 6], [5, 2, 0, 1, 4, 3, 7, 6], [5, 7, 3, 4, 2, 1, 0, 6], [6, 0, 1, 7, 3, 4, 5, 2], [3, 2, 7, 6, 4, 5, 1, 0], [3, 1, 7, 0, 6, 4, 2, 5], [6, 2, 4, 7, 0, 5, 3, 1], [0, 4, 6, 5, 7, 3, 1, 2]]
women_eng (1, 0, 3, 5, 6, 2, 7, 4)
men_eng [1, 0, 5, 2, 7, 3, 4, 6]


In [252]:
sadness=sadness_vectors(men_pref,women_pref,sm[k])
sadness

{'men_sadness': [1, 1, 0, 0, 0, 0, 0, 0],
 'women_sadness': [2, 2, 2, 6, 3, 6, 3, 1]}

In [253]:
men_sadness,women_sadnes=sadness["men_sadness"],sadness["women_sadness"]

## Sort marriages by women sadness

In [254]:
def sort_by_women_sadness(marriages):
    return sorted(marriages, key=lambda x: sum(sadness_vectors(men_pref,women_pref,x)["women_sadness"]))  # Sort based on the second element of each tuple


In [255]:
sm_by_women_sadness=sort_by_women_sadness(sm)
sm_by_women_sadness

[(1, 5, 7, 4, 2, 3, 6, 0),
 (1, 5, 7, 4, 6, 3, 2, 0),
 (1, 0, 7, 5, 2, 3, 6, 4),
 (1, 0, 7, 5, 6, 3, 2, 4),
 (0, 1, 7, 5, 2, 3, 6, 4),
 (0, 1, 7, 5, 6, 3, 2, 4),
 (1, 5, 3, 4, 6, 2, 7, 0),
 (1, 0, 3, 5, 6, 2, 7, 4),
 (0, 1, 3, 5, 6, 2, 7, 4)]

In [256]:
gale_shapley(men_pref,women_pref)["disponents_partners"]

[0, 1, 3, 5, 6, 2, 7, 4]

## Sort marriages by men sadness

In [257]:

def sort_by_men_sadness(marriages):
    return sorted(marriages, key=lambda x: sum(sadness_vectors(men_pref,women_pref,x)["men_sadness"]))  # Sort based on the second element of each tuple

In [258]:
sm_by_men_sadness=sort_by_men_sadness(sm)
sm_by_men_sadness

[(0, 1, 3, 5, 6, 2, 7, 4),
 (1, 0, 3, 5, 6, 2, 7, 4),
 (0, 1, 7, 5, 6, 3, 2, 4),
 (1, 5, 3, 4, 6, 2, 7, 0),
 (1, 0, 7, 5, 6, 3, 2, 4),
 (0, 1, 7, 5, 2, 3, 6, 4),
 (1, 0, 7, 5, 2, 3, 6, 4),
 (1, 5, 7, 4, 6, 3, 2, 0),
 (1, 5, 7, 4, 2, 3, 6, 0)]

In [259]:
gale_shapley(men_pref,women_pref)["disponents_partners"]

[0, 1, 3, 5, 6, 2, 7, 4]

## Sort by average sadness

In [260]:

def sort_by_average_sadness(marriages):
    return sorted(marriages, key=lambda x: sum(sadness_vectors(men_pref,women_pref,x)["men_sadness"])+sum(sadness_vectors(men_pref,women_pref,x)["women_sadness"]))  # Sort based on the second element of each tuple

In [261]:
sm_by_avg=sort_by_average_sadness(sm)
sm_by_avg

[(1, 0, 7, 5, 6, 3, 2, 4),
 (1, 5, 7, 4, 6, 3, 2, 0),
 (1, 0, 7, 5, 2, 3, 6, 4),
 (1, 5, 7, 4, 2, 3, 6, 0),
 (0, 1, 7, 5, 6, 3, 2, 4),
 (1, 0, 3, 5, 6, 2, 7, 4),
 (1, 5, 3, 4, 6, 2, 7, 0),
 (0, 1, 7, 5, 2, 3, 6, 4),
 (0, 1, 3, 5, 6, 2, 7, 4)]

In [262]:
gale_shapley(men_pref,women_pref)["disponents_partners"]

[0, 1, 3, 5, 6, 2, 7, 4]

## Sort by maximal sadness

In [263]:

def sort_by_maximal_sadness(marriages):
    return sorted(marriages, key=lambda x: sorted(sadness_vectors(men_pref,women_pref,x)["men_sadness"]+sadness_vectors(men_pref,women_pref,x)["women_sadness"],reverse=True))

In [264]:
sm_by_max=sort_by_maximal_sadness(sm)
sm_by_max

[(1, 5, 7, 4, 6, 3, 2, 0),
 (1, 5, 7, 4, 2, 3, 6, 0),
 (1, 0, 7, 5, 6, 3, 2, 4),
 (1, 0, 7, 5, 2, 3, 6, 4),
 (1, 5, 3, 4, 6, 2, 7, 0),
 (1, 0, 3, 5, 6, 2, 7, 4),
 (0, 1, 7, 5, 6, 3, 2, 4),
 (0, 1, 7, 5, 2, 3, 6, 4),
 (0, 1, 3, 5, 6, 2, 7, 4)]

In [265]:
for item in sm_by_max:
    x=sadness_vectors(men_pref,women_pref,item)
    
    print(x,sorted(x["women_sadness"]+x["men_sadness"],reverse=True))

{'men_sadness': [3, 1, 2, 1, 1, 1, 0, 3], 'women_sadness': [2, 0, 1, 5, 3, 0, 1, 0]} [5, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
{'men_sadness': [3, 1, 5, 1, 1, 1, 1, 3], 'women_sadness': [2, 0, 1, 5, 1, 0, 0, 0]} [5, 5, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
{'men_sadness': [1, 1, 2, 1, 0, 0, 0, 3], 'women_sadness': [2, 2, 1, 6, 3, 0, 1, 1]} [6, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
{'men_sadness': [1, 1, 5, 1, 0, 0, 1, 3], 'women_sadness': [2, 2, 1, 6, 1, 0, 0, 1]} [6, 5, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
{'men_sadness': [3, 1, 0, 0, 1, 1, 0, 0], 'women_sadness': [2, 0, 2, 5, 3, 6, 3, 0]} [6, 5, 3, 3, 3, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0]
{'men_sadness': [1, 1, 0, 0, 0, 0, 0, 0], 'women_sadness': [2, 2, 2, 6, 3, 6, 3, 1]} [6, 6, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0]
{'men_sadness': [0, 0, 2, 1, 0, 0, 0, 3], 'women_sadness': [6, 3, 1, 6, 3, 0, 1, 1]} [6, 6, 3, 3, 3, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
{'men_sadness': [0, 0, 5, 1, 0, 0, 1, 3], 'women_sadness': [6,

In [266]:
print(sm_by_men_sadness)
print(sm_by_women_sadness[::-1])

[(0, 1, 3, 5, 6, 2, 7, 4), (1, 0, 3, 5, 6, 2, 7, 4), (0, 1, 7, 5, 6, 3, 2, 4), (1, 5, 3, 4, 6, 2, 7, 0), (1, 0, 7, 5, 6, 3, 2, 4), (0, 1, 7, 5, 2, 3, 6, 4), (1, 0, 7, 5, 2, 3, 6, 4), (1, 5, 7, 4, 6, 3, 2, 0), (1, 5, 7, 4, 2, 3, 6, 0)]
[(0, 1, 3, 5, 6, 2, 7, 4), (1, 0, 3, 5, 6, 2, 7, 4), (1, 5, 3, 4, 6, 2, 7, 0), (0, 1, 7, 5, 6, 3, 2, 4), (0, 1, 7, 5, 2, 3, 6, 4), (1, 0, 7, 5, 6, 3, 2, 4), (1, 0, 7, 5, 2, 3, 6, 4), (1, 5, 7, 4, 6, 3, 2, 0), (1, 5, 7, 4, 2, 3, 6, 0)]


In [267]:
sm_by_avg==sm_by_max

False

### Create order 

In [268]:
def women_rather(x:list,y:list):### Takes two marriages x and y and returns whether or not women prefere x over y
    n=len(x)
    ans=True
    sadness_X=sadness_vectors(men_pref,women_pref,x)
    sadness_Y=sadness_vectors(men_pref,women_pref,y)
    for i in range(n):
        if sadness_X["women_sadness"][i]> sadness_Y["women_sadness"][i]:
            ans=False
            break
    return ans



In [269]:
M=[]
for marriage in sm_by_women_sadness:
    M.append([women_rather(marriage,item) for item in sm_by_women_sadness])
M
        

[[True, True, True, True, True, True, True, True, True],
 [False, True, False, True, False, True, True, True, True],
 [False, False, True, True, True, True, False, True, True],
 [False, False, False, True, False, True, False, True, True],
 [False, False, False, False, True, True, False, False, True],
 [False, False, False, False, False, True, False, False, True],
 [False, False, False, False, False, False, True, True, True],
 [False, False, False, False, False, False, False, True, True],
 [False, False, False, False, False, False, False, False, True]]

In [270]:
def men_rather(x:list,y:list):### Takes two marriages x and y and returns whether or not women prefere x over y
    n=len(x)
    ans=True
    sadness_X=sadness_vectors(men_pref,women_pref,x)
    sadness_Y=sadness_vectors(men_pref,women_pref,y)
    for i in range(n):
        if sadness_X["men_sadness"][i]> sadness_Y["men_sadness"][i]:
            ans=False
            break
    return ans

## Check that the two relationships define the same order

In [271]:
pairs_according_to_women={(A,B) for (A,B) in product(sm,sm) if women_rather(A,B)}
pairs_according_to_men={(B,A) for (A,B) in product(sm,sm) if men_rather(A,B)}

In [272]:
pairs_according_to_men==pairs_according_to_women

True

## The Join Operation (v)

In [273]:
def join(marriage_x,marriage_y):
    # The join operation takes two marriages and computes the join between them (the women choose the best of the two options)
    women_sadness_X=sadness_vectors(men_pref,women_pref,marriage_x)["women_sadness"]
    women_sadness_Y=sadness_vectors(men_pref,women_pref,marriage_y)["women_sadness"]
    ans=list(marriage_x)
    for i in range(len(marriage_x)):
        if women_sadness_X[i]>women_sadness_Y[i]:
            ans[i]=marriage_y[i]
    return tuple(ans)

In [274]:
count=[]
for (x,y) in product(sm,sm):
    if join(x,y) not in sm:
        count.append((x,y))
count
      

[]

## The Meet operation (^)

In [275]:
def meet(marriage_x,marriage_y):
    # The meet operation takes two marriages and computes the join between them (the men choose the best of the two options)
    men_sadness_X=sadness_vectors(men_pref,women_pref,marriage_x)["men_sadness"]
    men_sadness_Y=sadness_vectors(men_pref,women_pref,marriage_y)["men_sadness"]
    reversed_marriage_x=reverse(list(marriage_x))
    reversed_marriage_y=reverse(list(marriage_y))
    reversed_ans=reversed_marriage_x
    for i in range(len(marriage_x)):
        if men_sadness_X[i]>men_sadness_Y[i]:
            reversed_ans[i]=reversed_marriage_y[i]
    ans=tuple(reverse(reversed_ans))
    return ans

In [276]:
count=[]
for (x,y) in product(sm,sm):
    if meet(x,y) not in sm:
        count.append((x,y))
count

[]

## Check that X v (Y^Z)=(XvY)^(XvZ)

In [277]:
for x,y,z in product(sm,repeat=3):
    if join(x,meet(y,z))!=meet(join(x,y),join(x,z)):
        print("wtf")

In [278]:
len(SM)

8

## Check that X ^ (YvZ)=(X^Y)v(X^Z)

In [279]:
for x,y,z in product(sm,repeat=3):
    if meet(x,join(y,z))!=join(meet(x,y),meet(x,z)):
        print("wtf")

In [280]:
for x,y in product(sm,repeat=2):
    if not women_rather(join(x,y),x) or not women_rather(join(x,y),y) or not women_rather(x,meet(x,y)) or not women_rather(y,meet(x,y)):
        print("wtf")

## Check Birkoff's theorem

### Define the bijections

In [282]:

N=len(sm)
X=list(range(N))
reducibles=[tuple(gale_shapley(men_pref,women_pref)["disponents_partners"])]
for x,y in product(sm,sm):
    if not women_rather(x,y) and not women_rather(y,x):
        reducibles.append(join(x,y))
reducibles=[i for i in X if sm[i] in reducibles]
irreducibles=[i for i in X if i not in reducibles]
F=dict()
for marriage in sm:
    F[marriage]=tuple(sorted([x for x in irreducibles if women_rather(marriage,sm[x])]))
G=dict()
for item in list(F.keys()):
    G[F[item]]=item
SM=list(F.values())

In [283]:
SM

[(), (1, 2), (2,), (3,), (1, 2, 3), (2, 3), (3, 6), (1, 2, 3, 6), (2, 3, 6)]

In [286]:
for item in sm:
    if G[F[item]]!=item:
        print("wtf")

In [287]:
for item in SM:
    if F[G[item]]!=item:
        print("wtf")

### Define the join and meet operations in SM

In [288]:
def MEET(tuple_x,tuple_y):
    return tuple(sorted([x for x in X if x in tuple_x and x in tuple_y]))
def JOIN(tuple_x,tuple_y):
    return tuple(sorted([x for x in X if x in tuple_x or x in tuple_y]))


### Check that F is an isomorphism

In [289]:
for x,y in product(sm,sm):
    if F[join(x,y)]!=JOIN(F[x],F[y]) or F[meet(x,y)]!=MEET(F[x],F[y]):
        print("wtf")
        

In [290]:
SM

[(), (1, 2), (2,), (3,), (1, 2, 3), (2, 3), (3, 6), (1, 2, 3, 6), (2, 3, 6)]